# Skills to Salary Correlation
### What skills are required for higher-paying roles?

This project aims to see what skills pay off in terms of salary.

Goals:
- Discover what technical skills are required for higher-paying roles
- Organize data through different filters including region, remote-eligibility, experience requirements, etc.

### getting data

In [1]:
!git clone https://github.com/jsar2319/skills-salary-analysis.git

fatal: destination path 'skills-salary-analysis' already exists and is not an empty directory.


In [2]:
%cd skills-salary-analysis

/content/skills-salary-analysis


data from https://www.kaggle.com/datasets/mann14/global-ai-and-data-science-job-market-20202026/

### extract data

In [22]:
# import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [23]:
# load data into dataframes

jobs = pd.read_csv('data/ai_jobs.csv')
skills = pd.read_csv('data/skills_demand.csv')
titles = pd.read_csv('data/job_title_mapping.csv')

In [24]:
jobs.head(5)

,job_id,job_title,company_type,industry,country,city,remote_type,experience_level,min_experience_years,salary_min_usd,salary_max_usd,employment_type,posted_year,company_size
0,0IFD0TVBDIVU,MLOps Engineer,Research Lab,Education,Australia,Remote,Remote,Entry,0,56873,72223,Full-time,2023,Large
1,ZMF8MDD4V30T,Data Analyst,Startup,Education,Germany,Remote,Remote,Entry,0,54803,85599,Full-time,2024,Medium
2,CX1945NQ4FMY,MLOps Engineer,Research Lab,Tech,Canada,Remote,Remote,Senior,5,149980,175806,Full-time,2021,Large
3,QJ7YHL1C32OC,Applied Scientist,Research Lab,Healthcare,Australia,Remote,Remote,Entry,0,53483,86477,Full-time,2023,Medium
4,F0T0PVN9ER14,Machine Learning Engineer,Research Lab,Finance,Australia,Sydney,Hybrid,Mid,2,102977,127298,Full-time,2023,Large


In [25]:
skills.head(5)

,job_id,skill,skill_category,skill_level
0,E0IFD0TVBDIV,R,Programming,Advanced
1,E0IFD0TVBDIV,Computer Vision,ML,Basic
2,E0IFD0TVBDIV,NLP,ML,Intermediate
3,E0IFD0TVBDIV,SQL,Programming,Intermediate
4,ZMF8MDD4V30T,NLP,ML,Advanced


In [26]:
titles.head(5)

,job_title,standardized_title,role_category
0,Data Scientist,Data Scientist,Analytics
1,Machine Learning Engineer,Machine Learning Engineer,Engineering
2,AI Researcher,AI Researcher,Analytics
3,Data Analyst,Data Analyst,Analytics
4,Applied Scientist,Applied Scientist,Analytics


In [27]:
df_original = skills.merge(jobs, on='job_id')
df_original = df_original.merge(titles, on='job_title')
df_original.head(5)

,job_id,skill,skill_category,skill_level,job_title,company_type,industry,country,city,remote_type,experience_level,min_experience_years,salary_min_usd,salary_max_usd,employment_type,posted_year,company_size,standardized_title,role_category
0,ZMF8MDD4V30T,NLP,ML,Advanced,Data Analyst,Startup,Education,Germany,Remote,Remote,Entry,0,54803,85599,Full-time,2024,Medium,Data Analyst,Analytics
1,ZMF8MDD4V30T,Python,Programming,Basic,Data Analyst,Startup,Education,Germany,Remote,Remote,Entry,0,54803,85599,Full-time,2024,Medium,Data Analyst,Analytics
2,ZMF8MDD4V30T,AWS,Cloud,Advanced,Data Analyst,Startup,Education,Germany,Remote,Remote,Entry,0,54803,85599,Full-time,2024,Medium,Data Analyst,Analytics
3,ZMF8MDD4V30T,Azure,Cloud,Intermediate,Data Analyst,Startup,Education,Germany,Remote,Remote,Entry,0,54803,85599,Full-time,2024,Medium,Data Analyst,Analytics
4,CX1945NQ4FMY,Computer Vision,ML,Basic,MLOps Engineer,Research Lab,Tech,Canada,Remote,Remote,Senior,5,149980,175806,Full-time,2021,Large,MLOps Engineer,Engineering


###transform data

In [28]:
# print number of unique values per category
df_original.nunique()

,0
job_id,2050
skill,11
skill_category,3
skill_level,3
job_title,6
company_type,3
industry,5
country,6
city,24
remote_type,3


This hints that some columns are likely to be duplicates or are redundant. 'employment_type' only having one unique value across all records indicates that it is unnecessary. 'experience_level' and 'min_experience_years' both have 3 unique values, so they likely display the same data.

In [29]:
# show columns that match all records
def dependency_score(df, a, b):
    return df.groupby(a)[b].nunique().max()

def show_matching_columns(df):
  pairs = []
  cols = df.columns

  for i in range(len(cols)):
      for j in range(len(cols)):
          if i != j:
              score = dependency_score(df, cols[i], cols[j])
              pairs.append((cols[i], cols[j], score))

  dep_df = pd.DataFrame(pairs, columns=['A','B','max_B_per_A'])

  dep_df.query("max_B_per_A == 1")

  dep_df_copy = dep_df.query("max_B_per_A == 1")[["A", "B"]]

  # Merge the dataframe with itself swapping A and B
  symmetric = dep_df_copy.merge(dep_df_copy, left_on=["A", "B"], right_on=["B", "A"])

  # Clean up to show just one pair per match
  symmetric_pairs = symmetric[["A_x", "B_x"]].rename(
      columns={"A_x": "A", "B_x": "B"}
  )

  # Canonicalize pairs (A,B) == (B,A)
  symmetric_pairs[["A", "B"]] = np.sort(symmetric_pairs[["A", "B"]], axis=1)

  # Drop duplicates
  unique_pairs = symmetric_pairs.drop_duplicates().reset_index(drop=True)

  print(unique_pairs)

show_matching_columns(df_original)

                  A                     B
0         job_title    standardized_title
1  experience_level  min_experience_years


In [30]:
# drop unnecessary/redundant columns
df = df_original.copy()
df.drop(columns=['employment_type', 'standardized_title', 'min_experience_years'], inplace=True)

In [32]:
df.head()

,job_id,skill,skill_category,skill_level,job_title,company_type,industry,country,city,remote_type,experience_level,salary_min_usd,salary_max_usd,posted_year,company_size,role_category,salary_avg
0,ZMF8MDD4V30T,NLP,ML,Advanced,Data Analyst,Startup,Education,Germany,Remote,Remote,Entry,54803,85599,2024,Medium,Analytics,70201.0
1,ZMF8MDD4V30T,Python,Programming,Basic,Data Analyst,Startup,Education,Germany,Remote,Remote,Entry,54803,85599,2024,Medium,Analytics,70201.0
2,ZMF8MDD4V30T,AWS,Cloud,Advanced,Data Analyst,Startup,Education,Germany,Remote,Remote,Entry,54803,85599,2024,Medium,Analytics,70201.0
3,ZMF8MDD4V30T,Azure,Cloud,Intermediate,Data Analyst,Startup,Education,Germany,Remote,Remote,Entry,54803,85599,2024,Medium,Analytics,70201.0
4,CX1945NQ4FMY,Computer Vision,ML,Basic,MLOps Engineer,Research Lab,Tech,Canada,Remote,Remote,Senior,149980,175806,2021,Large,Engineering,162893.0


In [31]:
# feature engineering
df['salary_avg'] = (df.salary_min_usd + df.salary_max_usd) / 2